# Test 19: Discriminant test for the origin of a p-fold sample

## Objective

- This is very similar to `Test 18`
    - In `Test 18`, we had 2 groups of values and some $n$ observations of $k$ features in each group
    - We wanted to know, are the 2 groups sufficiently different such that we can reject the null that they are the same

- In this case, we have an observation 

## Assumptions

## Method

## Example